In [3]:
import cv2
import mediapipe as mp
import numpy as np
import csv

import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [7]:
cap = cv2.VideoCapture (0)
height= cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width= cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get (cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('hand_exces_on_hip.mp4', cv2.VideoWriter_fourcc('P','I', 'M','1'), fps, (int(width), int(height)))
while cap.isOpened():
    ret, frame = cap.read()
    try:
        cv2.imshow('hand_exces', frame)
        videoWriter.write(frame)
    except  Exception as e:
        break
    if cv2.waitKey (10) & 0xFF == ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

In [8]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [9]:
with open('feature_hand_on_hip.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [10]:
def export_landmark(results, action):
    try:
        keypoints = [action]  # Initialize the list with the action label

        # Extract pose landmarks and append them to the keypoints list
        for landmark in results.pose_landmarks.landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])

        # Open the CSV file in 'a' (append) mode and write the keypoints
        with open('feature_hand_on_hip.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(f"Error in export_landmark: {e}")

In [11]:

# Create a VideoCapture object to open the video file
video_path = 'C:/Projects/FinalProject/excesise_proj/hand_on_hip/hand_exces_on_hip.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Initialize a variable to keep track of the current pose
current_pose = None

# Create a dictionary to map keys to pose labels
key_to_pose = {
    ord('a'): 'on',
    ord('s'): 'off',
    
}

# Main processing loop
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break  # Exit the loop when there are no more frames

        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform pose detection
        results = pose.process(image)

        # Convert back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and lines on the frame
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Check for key presses and export landmarks
        key = cv2.waitKey(1)
        if key in key_to_pose:
            current_pose = key_to_pose[key]
            #print(current_pose)
            export_landmark(results, current_pose)

        # Display the frame
        cv2.imshow('webcam feed', image)

        # Check for 'q' key to quit
        if key == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [12]:
df = pd.read_csv('C:/Projects/FinalProject/excesise_proj/hand_on_hip/feature_hand_on_hip.csv')

In [13]:
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,on,0.668563,0.120181,-0.498283,0.999853,0.677200,0.101272,-0.474266,0.999744,0.681991,...,0.220430,0.357438,0.659741,0.958414,-0.036295,0.566453,0.598400,0.961163,0.062490,0.534367
1,on,0.669933,0.120587,-0.532538,0.999861,0.678311,0.101530,-0.508745,0.999755,0.683089,...,0.235841,0.363367,0.660096,0.958420,-0.033820,0.577249,0.598261,0.959520,0.079258,0.538780
2,on,0.671389,0.122678,-0.553765,0.999864,0.679861,0.102817,-0.530678,0.999754,0.684745,...,0.227264,0.363772,0.660215,0.955312,-0.016064,0.581122,0.597840,0.953487,0.072114,0.541281
3,on,0.673790,0.124221,-0.529900,0.999870,0.682584,0.104307,-0.506338,0.999752,0.687715,...,0.228921,0.364003,0.663894,0.944897,0.012726,0.577325,0.597025,0.945990,0.077319,0.540129
4,on,0.673591,0.125733,-0.521999,0.999869,0.682360,0.105551,-0.498352,0.999746,0.687519,...,0.225348,0.360672,0.666085,0.945255,0.006127,0.568416,0.590295,0.947384,0.073001,0.530798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,on,0.663179,0.120586,-0.623610,0.999846,0.672716,0.101396,-0.599323,0.999622,0.677462,...,0.310073,0.290220,0.674162,0.954102,0.102706,0.477991,0.600472,0.975085,0.138855,0.450377
473,off,0.673728,0.122782,-0.644837,0.999881,0.683143,0.101419,-0.622487,0.999704,0.687944,...,0.294637,0.263017,0.664392,0.956695,0.058568,0.452546,0.616721,0.963314,0.116051,0.421199
474,off,0.678257,0.123553,-0.644783,0.999890,0.687125,0.101980,-0.622168,0.999725,0.691995,...,0.287654,0.269500,0.664797,0.959535,0.055172,0.468205,0.622349,0.978066,0.114921,0.435484
475,off,0.679629,0.125702,-0.636785,0.999899,0.688351,0.103459,-0.615275,0.999745,0.693395,...,0.237785,0.290524,0.665382,0.959352,0.053545,0.494074,0.642298,0.972466,0.065092,0.463701


In [14]:
x = df.drop('class', axis=1)
y=df['class']

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=1234)

In [16]:
model = LogisticRegression()

model.fit(x_train, y_train)

LogisticRegression()

In [17]:
y_pred = model.predict(x_test)

In [1]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

NameError: name 'accuracy_score' is not defined

In [19]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
state = ''

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            x = pd.DataFrame([row], columns=landmarks[1:])
            state_class = model.predict(x)[0]
            state_prob = model.predict_proba(x)[0]
            #print(state_class, state_prob)

            if state_class == 'on' and state_prob[state_prob.argmax()] >= 0.7:
                state = 'on'
            elif state == 'on' and state_class == 'off' and state_prob[state_prob.argmax()] >= 0.7:
                state = 'off'
                

            cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)

            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, state_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, str(round(state_prob[np.argmax(state_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
        except Exception as e:
            print(f"Error: {e}")

        cv2.imshow('webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'


In [20]:
import pickle

hand_on_hip = 'C:/Users/shana/OneDrive/Desktop/excesise_proj/hand_on_hip/hand_on_hip.pkl'

with open(hand_on_hip, 'wb') as file:
    pickle.dump(model, file)
print(f"Model saved as {hand_on_hip}")

Model saved as C:/Users/shana/OneDrive/Desktop/excesise_proj/hand_on_hip/hand_on_hip.pkl


In [21]:
with open(hand_on_hip, 'rb') as file:
    hand_on_hip = pickle.load(file)

In [22]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
state = ''

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            x = pd.DataFrame([row], columns=landmarks[1:])
            state_class = hand_on_hip.predict(x)[0]
            state_prob = hand_on_hip.predict_proba(x)[0]
            #print(state_class, state_prob)

            if state_class == 'on' and state_prob[state_prob.argmax()] >= 0.7:
                state = 'on'
            elif state == 'on' and state_class == 'off' and state_prob[state_prob.argmax()] >= 0.7:
                state = 'off'
                

            cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)

            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, state_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, str(round(state_prob[np.argmax(state_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
        except Exception as e:
            print(f"Error: {e}")

        cv2.imshow('webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan